# Gemini models in  SAP AI Core

Using direct API calls with Vertex AI format.

## 1. Setup

In [19]:
import os
import json
import requests


## 2. Set Credentials

In [33]:
# Path to your AI Core service key JSON file
service_key_path = "../aicore-service-key.json"

# Load service key
with open(service_key_path, 'r') as f:
    service_key = json.load(f)

# Set environment variables
os.environ['AICORE_AUTH_URL'] = service_key['url']
os.environ['AICORE_CLIENT_ID'] = service_key['clientid']
os.environ['AICORE_CLIENT_SECRET'] = service_key['clientsecret']
os.environ['AICORE_RESOURCE_GROUP'] = 'default'

# Parse serviceurls - it might be a string or already a dict
serviceurls = service_key['serviceurls']
if isinstance(serviceurls, str):
    serviceurls = json.loads(serviceurls)
os.environ['AICORE_BASE_URL'] = serviceurls['AI_API_URL']
base_url = os.environ['AICORE_BASE_URL']

print("✓ AI Core credentials configured")


✓ AI Core credentials configured


## 3. Get Auth Token

In [27]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

# Get auth headers
proxy = get_proxy_client('gen-ai-hub')
headers = proxy.get_request_header()

print('✓ Auth token obtained')


✓ Auth token obtained


## 3b. Check for Image Generation Models


In [29]:
# List all deployments and check for image generation models
deployments = proxy.get_deployments()

print('All Available Deployments:')
print('='*80)

image_models = []

for d in deployments:
    model_name = d.model_name.lower()
    if "gemini" in model_name:
        print(f"\nModel: {d.model_name}")
        print(f"  Deployment ID: {d.deployment_id}")
        
        # Check if it's an image generation model
        if any(x in model_name for x in ['dall-e', 'dalle', 'stable-diffusion', 'sd-', 'imagen']):
            image_models.append(d)
            print("  ✓ IMAGE GENERATION MODEL")
        else:
            
            print("  📝 Text/Analysis model")

    print('\n' + '='*80)
    print(f'\nSummary: {len(image_models)} image generation model(s) found')

    if image_models:
        print('\n✓ Image generation models:')
        for m in image_models:
            print(f'  - {m.model_name} (ID: {m.deployment_id})')
    else:
        print('\n⚠️  No image generation models deployed in your SAP AI Core')


All Available Deployments:


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.5-pro
  Deployment ID: d6ae523ed14c6cc3
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.0-flash
  Deployment ID: d83ad32cbda0399d
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generati

## 4. Call Gemini 2.5 Pro (Vertex AI Format)

In [34]:
# Gemini 2.5 Pro deployment
DEPLOYMENT_ID = 'd6ae523ed14c6cc3'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID}/models/gemini-2.5-pro:generateContent'

# Vertex AI format payload
payload = {
    'contents': [
        {
            'role': 'user',
            'parts': [{'text': 'can you generate the cat image'}]
        }
    ],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 10000
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Success!')
    print(result)
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')


✓ Success!
{'candidates': [{'avgLogprobs': -0.6888435347908004, 'content': {'parts': [{'text': 'Of course!\n\nAs a text-based AI, I can\'t generate a visual image file directly. However, I can do the next best thing! I can either:\n\n1.  **Describe a cat image in detail** for you to use with an AI image generator (like Midjourney, DALL-E, or Stable Diffusion).\n2.  **Create a cat for you using text characters** (ASCII art).\n\nWhich would you prefer? Here are both options for you!\n\n---\n\n### 1. Detailed Descriptions for an AI Image Generator\n\nCopy and paste one of these prompts into an AI image tool to get a fantastic picture.\n\n**Prompt for a Photorealistic Cat:**\n> A highly detailed, photorealistic close-up of a fluffy Siberian cat sleeping in a patch of sunlight on a wooden floor. The cat\'s long, silver-tabby fur is soft and distinct. Dust motes dance in the warm, golden sunbeam. The focus is sharp on the cat\'s face, showing its peaceful expression and twitching whiskers. T

## 4b. Test Image Generation with Gemini 2.0 Flash


In [35]:
# Test if Gemini 2.0 Flash can generate images
DEPLOYMENT_ID_2_0 = 'd83ad32cbda0399d'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:generateContent'

# Try to ask it to generate an image
payload = {
    'contents': [
        {
            'role': 'user',
            'parts': [{'text': 'can you generate a cat image'}]
        }
    ],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 1000
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Success!')
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')
    
print('\n' + '='*80)
print('Note: Gemini models (both 2.5 Pro and 2.0 Flash) can ANALYZE images')
print('but cannot GENERATE images. They are text and image understanding models.')
print('For image generation, you would need models like DALL-E or Stable Diffusion.')


✓ Success!

Response:
I am unable to generate images directly. I am a text-based AI.

However, I can give you some ideas of what to search for online, or suggest prompts for an image generation tool if you have access to one. For example:

**Ideas for Image Searches:**

*   "Cute kitten"
*   "Sleeping cat"
*   "Funny cat meme"
*   "Cat portrait"
*   "Cartoon cat"

**Prompts for Image Generation Tools:**

*   "A fluffy ginger cat sitting in a sunbeam"
*   "A sleek black cat with green eyes staring intensely"
*   "A whimsical illustration of a cat wearing a tiny hat"
*   "A photorealistic image of a tabby cat curled up on a couch"
*   "A pixel art cat sprite"

I hope this helps you find the purr-fect cat image! 🐱


Note: Gemini models (both 2.5 Pro and 2.0 Flash) can ANALYZE images
but cannot GENERATE images. They are text and image understanding models.
For image generation, you would need models like DALL-E or Stable Diffusion.


## 5. Helper Function

In [31]:
def ask_gemini(prompt, temperature=0.7, max_tokens=500, model='2.5-pro'):
    """Call Gemini via Vertex AI format"""
    # Choose deployment
    deployments = {
        '2.5-pro': 'd6ae523ed14c6cc3',
        '2.0-flash': 'd83ad32cbda0399d'
    }
    deployment_id = deployments.get(model, deployments['2.5-pro'])
    model_name = 'gemini-2.5-pro' if model == '2.5-pro' else 'gemini-2.0-flash'
    
    url = f'{AI_API_URL}/v2/inference/deployments/{deployment_id}/models/{model_name}:generateContent'
    
    payload = {
        'contents': [{
            'role': 'user',
            'parts': [{'text': prompt}]
        }],
        'generationConfig': {
            'temperature': temperature,
            'maxOutputTokens': max_tokens
        }
    }
    
    resp = requests.post(url, headers=headers, json=payload)
    
    if resp.status_code == 200:
        return resp.json()['candidates'][0]['content']['parts'][0]['text']
    raise Exception(f'Error {resp.status_code}: {resp.text}')

# Test with 2.5 Pro
print('Testing Gemini 2.5 Pro:')
print(ask_gemini('Say hello!', model='2.5-pro'))

print('\nTesting Gemini 2.0 Flash:')
print(ask_gemini('Say hello!', model='2.0-flash'))


Testing Gemini 2.5 Pro:
Hello there! 👋

How can I help you today?

Testing Gemini 2.0 Flash:
Hello! How can I help you today?



## Gemini 2.0 Flash - All Available Endpoints


### 1. :generateContent (Standard - Synchronous)


In [ ]:
# Endpoint 1: Standard generateContent
DEPLOYMENT_ID_2_0 = 'd83ad32cbda0399d'
url = f'{AI_API_URL}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:generateContent'

payload = {
    'contents': [{
        'role': 'user',
        'parts': [{'text': 'Explain quantum computing in one sentence.'}]
    }],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 100
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Standard Generation Success!')
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')


### 2. :streamGenerateContent (Streaming - Real-time)


In [ ]:
# Endpoint 2: Streaming generateContent
url = f'{AI_API_URL}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:streamGenerateContent'

payload = {
    'contents': [{
        'role': 'user',
        'parts': [{'text': 'Write a short poem about AI.'}]
    }],
    'generationConfig': {
        'temperature': 0.9,
        'maxOutputTokens': 200
    }
}

print('✓ Streaming Response:')
print('-' * 60)

response = requests.post(url, headers=headers, json=payload, stream=True)

if response.status_code == 200:
    # Process streaming response
    import json
    for line in response.iter_lines():
        if line:
            try:
                # Parse each JSON chunk
                chunk = json.loads(line.decode('utf-8'))
                if 'candidates' in chunk and len(chunk['candidates']) > 0:
                    text = chunk['candidates'][0]['content']['parts'][0]['text']
                    print(text, end='', flush=True)
            except json.JSONDecodeError:
                continue
    print('\n' + '-' * 60)
    print('✓ Streaming complete!')
else:
    print(f'Error {response.status_code}: {response.text}')


### 3. :countTokens (Token Counting)


In [ ]:
# Endpoint 3: Count tokens in a prompt
url = f'{AI_API_URL}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:countTokens'

test_prompt = """
Write a comprehensive analysis of the impact of artificial intelligence 
on modern business operations, including specific examples from finance, 
healthcare, and manufacturing sectors.
"""

payload = {
    'contents': [{
        'role': 'user',
        'parts': [{'text': test_prompt}]
    }]
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Token Count Success!')
    print(f"\nPrompt: {test_prompt[:100]}...")
    print(f"\nTotal Tokens: {result.get('totalTokens', 'N/A')}")
    
    # Some implementations return more details
    if 'tokensPerCandidate' in result:
        print(f"Tokens per candidate: {result['tokensPerCandidate']}")
else:
    print(f'Error {response.status_code}: {response.text}')


### 4. :embedContent (Embeddings - for semantic search)


In [ ]:
# Endpoint 4: Generate embeddings (if supported)
# Note: Not all Gemini models support embeddings. This may require a specific embedding model.
url = f'{AI_API_URL}/v2/inference/deployments/{DEPLOYMENT_ID_2_0}/models/gemini-2.0-flash:embedContent'

texts_to_embed = [
    "Artificial intelligence is transforming business",
    "Machine learning powers modern applications",
    "SAP AI Core enables AI deployment"
]

payload = {
    'content': {
        'parts': [{'text': texts_to_embed[0]}]
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Embedding Generation Success!')
    
    if 'embedding' in result:
        embedding = result['embedding'].get('values', [])
        print(f"\nText: '{texts_to_embed[0]}'")
        print(f"Embedding dimensions: {len(embedding)}")
        print(f"First 10 values: {embedding[:10]}")
    else:
        print(f"Response: {result}")
else:
    print(f'⚠️  Embeddings may not be supported by gemini-2.0-flash')
    print(f'Error {response.status_code}: {response.text}')
    print('\nNote: For embeddings, you may need to use a dedicated embedding model')
    print('like text-embedding-004 or textembedding-gecko if available in your deployment.')


### Summary: All Gemini 2.0 Flash Endpoints

| Endpoint | Purpose | URL Pattern |
|----------|---------|-------------|
| `:generateContent` | Synchronous text generation | `.../gemini-2.0-flash:generateContent` |
| `:streamGenerateContent` | Real-time streaming responses | `.../gemini-2.0-flash:streamGenerateContent` |
| `:countTokens` | Count tokens in prompts | `.../gemini-2.0-flash:countTokens` |
| `:embedContent` | Generate embeddings (may not be supported) | `.../gemini-2.0-flash:embedContent` |


### Helper Functions for All Endpoints


In [ ]:
class GeminiClient:
    """Helper class for all Gemini 2.0 Flash endpoints"""
    
    def __init__(self, deployment_id='d83ad32cbda0399d', base_url=AI_API_URL, headers=None):
        self.deployment_id = deployment_id
        self.base_url = base_url
        self.headers = headers or proxy.get_request_header()
        self.model_name = 'gemini-2.0-flash'
    
    def generate(self, prompt, temperature=0.7, max_tokens=1024):
        """Standard synchronous generation"""
        url = f'{self.base_url}/v2/inference/deployments/{self.deployment_id}/models/{self.model_name}:generateContent'
        
        payload = {
            'contents': [{'role': 'user', 'parts': [{'text': prompt}]}],
            'generationConfig': {
                'temperature': temperature,
                'maxOutputTokens': max_tokens
            }
        }
        
        response = requests.post(url, headers=self.headers, json=payload)
        
        if response.status_code == 200:
            return response.json()['candidates'][0]['content']['parts'][0]['text']
        else:
            raise Exception(f'Error {response.status_code}: {response.text}')
    
    def stream_generate(self, prompt, temperature=0.7, max_tokens=1024):
        """Streaming generation"""
        url = f'{self.base_url}/v2/inference/deployments/{self.deployment_id}/models/{self.model_name}:streamGenerateContent'
        
        payload = {
            'contents': [{'role': 'user', 'parts': [{'text': prompt}]}],
            'generationConfig': {
                'temperature': temperature,
                'maxOutputTokens': max_tokens
            }
        }
        
        response = requests.post(url, headers=self.headers, json=payload, stream=True)
        
        if response.status_code == 200:
            import json
            for line in response.iter_lines():
                if line:
                    try:
                        chunk = json.loads(line.decode('utf-8'))
                        if 'candidates' in chunk and len(chunk['candidates']) > 0:
                            yield chunk['candidates'][0]['content']['parts'][0]['text']
                    except json.JSONDecodeError:
                        continue
        else:
            raise Exception(f'Error {response.status_code}: {response.text}')
    
    def count_tokens(self, prompt):
        """Count tokens in a prompt"""
        url = f'{self.base_url}/v2/inference/deployments/{self.deployment_id}/models/{self.model_name}:countTokens'
        
        payload = {
            'contents': [{'role': 'user', 'parts': [{'text': prompt}]}]
        }
        
        response = requests.post(url, headers=self.headers, json=payload)
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f'Error {response.status_code}: {response.text}')
    
    def embed(self, text):
        """Generate embeddings (may not be supported)"""
        url = f'{self.base_url}/v2/inference/deployments/{self.deployment_id}/models/{self.model_name}:embedContent'
        
        payload = {
            'content': {'parts': [{'text': text}]}
        }
        
        response = requests.post(url, headers=self.headers, json=payload)
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f'Error {response.status_code}: {response.text}')


# Create client instance
gemini = GeminiClient()

print('✓ GeminiClient initialized!')
print('\nUsage examples:')
print('  result = gemini.generate("Your prompt here")')
print('  for chunk in gemini.stream_generate("Your prompt"): print(chunk, end="")')
print('  tokens = gemini.count_tokens("Your prompt")')
print('  embedding = gemini.embed("Your text")')


### Demo: Using the GeminiClient Helper


In [ ]:
# Demo 1: Standard generation
print('=== Demo 1: Standard Generation ===')
response = gemini.generate('Explain SAP AI Core in 2 sentences.', max_tokens=100)
print(response)
print()

# Demo 2: Streaming generation
print('=== Demo 2: Streaming Generation ===')
print('Response: ', end='')
for chunk in gemini.stream_generate('Count from 1 to 5 with descriptions.', max_tokens=150):
    print(chunk, end='', flush=True)
print('\n')

# Demo 3: Token counting
print('=== Demo 3: Token Counting ===')
test_text = "How many tokens are in this sentence?"
token_info = gemini.count_tokens(test_text)
print(f'Text: "{test_text}"')
print(f'Token count: {token_info}')
print()

# Demo 4: Embeddings (may fail if not supported)
print('=== Demo 4: Embeddings ===')
try:
    embedding_result = gemini.embed('SAP AI Core deployment')
    print(f'✓ Embedding generated: {embedding_result}')
except Exception as e:
    print(f'⚠️  Embeddings not supported: {e}')


## 6. Multi-turn Conversation

In [ ]:
# Conversation with history
DEPLOYMENT_ID = 'd6ae523ed14c6cc3'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID}/models/gemini-2.5-pro:generateContent'

# Build conversation
contents = [
    {'role': 'user', 'parts': [{'text': 'What is SAP BTP?'}]}
]

resp = requests.post(url, headers=headers, json={'contents': contents})
reply1 = resp.json()['candidates'][0]['content']['parts'][0]['text']
print('User: What is SAP BTP?')
print(f'AI: {reply1}\n')

# Continue conversation
contents.append({'role': 'model', 'parts': [{'text': reply1}]})
contents.append({'role': 'user', 'parts': [{'text': 'How does it relate to AI Core?'}]})

resp = requests.post(url, headers=headers, json={'contents': contents})
reply2 = resp.json()['candidates'][0]['content']['parts'][0]['text']
print('User: How does it relate to AI Core?')
print(f'AI: {reply2}')


User: What is SAP BTP?
AI: Of course. Let's break down SAP BTP in a clear, structured way.

### The Simple Analogy

Imagine your core SAP system (like S/4HANA) is a brand-new, high-end smartphone. It's powerful and does its main job—running your business—exceptionally well.

Now, you want to add new, custom features: a special app for your sales team, a unique dashboard for your CEO, or a way to connect to a new supplier's system.

You have two choices:
1.  **Jailbreak the Phone:** Hack the phone's core operating system. This might work, but it's risky, makes future updates a nightmare, and could break the phone.
2.  **Use the App Store and Developer Tools:** Use the official tools and platform provided by the phone manufacturer to build and run new apps. These apps work seamlessly with the phone but run separately, keeping the core operating system clean, stable, and easy to update.

**SAP BTP is the "App Store and Developer Tools" for your SAP landscape.** It's a platform that lets y